# Scrapy + Scraper API

In [ ]:
pip install scrapy

     |████████████████████████████████| 245kB 6.4MB/s 
     |████████████████████████████████| 3.1MB 8.9MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 245kB 31.4MB/s 
     |████████████████████████████████| 2.6MB 36.1MB/s 
     |████████████████████████████████| 3.2MB 42.0MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=263d9553a63effa8aa16096743e466fe559aa3ee074541c3d23df99df5e85c20
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7766 sha256=562ce797de3cf10fbdc8554dc0f37462eca764fb22ec59d3825352f060e02c8f
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatche

In [ ]:
import scrapy
from urllib.parse import urlencode
from urllib.parse import urlparse
import json
from datetime import datetime
API_KEY = 'fff2df1787bf81bfe98277920f9a9fcb'

In [ ]:
def get_url(url):
   payload = {'api_key': API_KEY, 'url': url, 'autoparse': 'true', 'country_code': 'fr'}
   proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
   return proxy_url

def create_google_url(query, site=''):
   google_dict = {'q': query, 'num': 100, }
   if site:
       web = urlparse(site).netloc
       google_dict['as_sitesearch'] = web
       return 'http://www.google.com/search?' + urlencode(google_dict)
   return 'http://www.google.com/search?' + urlencode(google_dict)

class GoogleSpider(scrapy.Spider):
   name = 'google'
   #allowed_domains = ['api.scraperapi.com']
   #custom_settings = {'ROBOTSTXT_OBEY': False, 'LOG_LEVEL': 'INFO', 'CONCURRENT_REQUESTS_PER_DOMAIN': 5, 'RETRY_TIMES': 0}
   allowed_domains = ['www.google.com']
   custom_settings = {'ROBOTSTXT_OBEY': False, 'LOG_LEVEL': 'INFO', 'CONCURRENT_REQUESTS_PER_DOMAIN': 5, 'RETRY_TIMES': 0}

   def start_requests(self):
       queries = ['innovation'] 
       for query in queries:
           url = create_google_url(query)
           yield scrapy.Request(get_url(url), callback=self.parse, meta={'pos': 0})

   def parse(self, response):
       di = json.loads(response.text)
       pos = response.meta['pos']
       dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
       for result in di['organic_results']:
           title = result['title']
           snippet = result['snippet']
           link = result['link']
           item = {'title': title, 'snippet': snippet, 'link': link, 'position': pos, 'date': dt}
           pos += 1
           yield item
       next_page = di['pagination']['nextPageUrl']
       if next_page:
           yield print(scrapy.Request(get_url(next_page), callback=self.parse, meta={'pos': pos}).text)

In [ ]:
if __name__ == '__main__':
    result = list(GoogleSpider().start_requests())

In [ ]:
result

[<GET http://api.scraperapi.com/?api_key=fff2df1787bf81bfe98277920f9a9fcb&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr>]

In [ ]:
link = str(result[0])[5:-1]
link

'http://api.scraperapi.com/?api_key=fff2df1787bf81bfe98277920f9a9fcb&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr'

In [ ]:
# On importe la fonction 'get' (téléchargement) de 'requests' 
# Et la classe 'Selector' (Parsing) de 'scrapy'
from requests import get
from scrapy import Selector
# Lien de la page à scraper
url = link
response = get(url)
source = None # Le code source de la page 
if response.status_code == 200 :
    # Si la requete s'est bien passee
    source = response.text

In [ ]:
source

'{"search_information":{"total_results":749,"time_taken_displayed":0,"query_displayed":"innovation"},"ads":[],"knowledge_graph":{"title":"","description":"","source":{"name":""},"related":[],"related_link":null,"social_media":[],"see_more_about":[{"title":"Recherches associées","link":"https://www.google.com/search?num=100&gl=FR&q=Innovation&stick=H4sIAAAAAAAAAONgFuLWT9c3NDKyTMkuM1NC5mgJZydb6Sfn5-bm51kVZ6aklidWFi9i5fLMy8svSyzJzM_bwcoIALUghDRBAAAA&sa=X&ved=2ahUKEwiV1tvbx4TuAhUM7awKHYwTAycQMSgAMIsBegQIEBAB"},{"title":"Voir d\'autres éléments (plus de\xa010)","link":"https://www.google.com/search?num=100&gl=FR&q=Innovation&stick=H4sIAAAAAAAAAONgFuLWT9c3NDKyTMkuM1NC5mgJZydb6Sfn5-bm51kVZ6aklidWFi9i5fLMy8svSyzJzM_bwcoIALUghDRBAAAA&sa=X&ved=2ahUKEwiV1tvbx4TuAhUM7awKHYwTAycQzTooATCLAXoECBAQAg"},{"title":"Créativité","link":"https://www.google.com/search?num=100&gl=FR&q=Cr%C3%A9ativit%C3%A9&stick=H4sIAAAAAAAAAONgFuLWT9c3NDKyTMkuM1Pi1M_VNzA0MkjL0-Jzzs_Nzc8LzkxJLU-sLF7EyuNcdHhlYklmWWbJ4ZU7WBkBixI